In [ ]:
import numpy as np
import scipy.stats
import pandas as pd
import time as time
import os
import glob
import constants

# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib as mpl
mpl.rc('mathtext',fontset='stixsans')
mpl.rc('figure', facecolor="white")
#matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
#import matplotlib.colors as colors
# %matplotlib notebook

from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

# Define cosmology
cosmo = constants.COSMOLOGY

PiBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins23_pi_0-30hMpc.txt')
SigBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins10_sigma_0-30hMpc.txt')

PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)

PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data

# Convert bin boundaries from Mpc/h to Mpc
PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])

print('Pi bin edges in Mpc:')
print(PiEdges)
print('Sigma bin edges in Mpc:')
print(SigEdges)

GAL_DIR = os.path.join(constants.XCORR_DIR_BASE, 'mock', 'gal')
MOCK_XCORR_DIR = os.path.join(constants.XCORR_DIR_BASE, 'mock', 'crosscorr')

SIM_MPCH_LEN = 250 # Mpc/h

SURVEY_NAME = 'zDeep'

# gnum = 1
# absnum= 64

# PiBound = (min(PiEdges), max(PiEdges) )

# # Read in galaxies to provide counts in plot titles
# galfil = os.path.join(GAL_DIR, 'cat_galmock_nonuniq_{0:03d}.dat'.format(gnum))
# gal = ascii.read(galfil,format='ipac')
# #print(gal.columns)

# gal_3d = gal[gal['source'] == '3DHST']
# gal_zD = gal[gal['source'] == 'zDeep']
# gal_mosdef = gal[gal['source'] == 'MOSDEF']
# gal_vuds = gal[gal['source']=='VUDS']
# gal_clamato=gal[gal['source']=='CLAMATO']

In [ ]:
# We will create mocks over these number of distinct fields
nmock_x = 7
nmock_y = 9
nmocks = nmock_x * nmock_y

nabs = 3
ngal = 10

width_x = 34. # Mpc/h
width_y = 28. # Mpc/h

# Generate tuples indicating the field boundaries and center
xlow_field = np.arange(nmock_x)*width_x
xhi_field = (np.arange(nmock_x) + 1.)*width_x 
xcen_field = np.ndarray.tolist((xlow_field+xhi_field)/2.)
xlow_field = np.ndarray.tolist(xlow_field)
xhi_field = np.ndarray.tolist(xhi_field)
xboundaries = list(zip(xlow_field, xcen_field, xhi_field))

ylow_field = np.arange(nmock_y) * width_y
yhi_field  = (np.arange(nmock_y)+1.)* width_y
ycen_field = np.ndarray.tolist((ylow_field+yhi_field)/2.)
ylow_field = np.ndarray.tolist(ylow_field)
yhi_field = np.ndarray.tolist(yhi_field)
yboundaries = list(zip(ylow_field, ycen_field, yhi_field))

In [ ]:
# Load in the full Bolshoi-Planck z=0 halo catalog.
full_halo_cat = pd.read_csv('/global/homes/b/bzh/projectdir/bzh/BolshoiP/full-halo-cat/hlist_1.00231.list', header=None, comment='#', delim_whitespace=True)
full_halo_cat = full_halo_cat.rename(columns={1: 'id', 10: 'mvir_snap', 17: 'x_snap', 18: 'y_snap', 19: 'z_snap'})

# HALO_MASS_THRESHOLD = 10**14.3
HALO_MASS_THRESHOLD = 10**14

print(f'# halos before cut is {len(full_halo_cat)}')
full_halo_cat = full_halo_cat.loc[full_halo_cat.mvir_snap >= HALO_MASS_THRESHOLD]
print(f'# halos after cut is {len(full_halo_cat)}')

In [ ]:
massive_halo_counts = []
# Account for the fact that some footprints may extend outside the simulation
massive_halo_areal_dens = []
# Sum up the cross-correlation from sigma >15 Mpc (marginalized over pi) as a measure of continuum correlation.
cont_xcorr_means = []
cont_xcorr_stds = []

mock_ctr = 1

for (xlow_tmp, xcen_tmp, xhi_tmp) in xboundaries:
    for (ylow_tmp, ycen_tmp, yhi_tmp) in yboundaries:
        if xhi_tmp > SIM_MPCH_LEN:
            print(f'Trimming x-high boundary from {xhi_tmp} Mpc/h to sim length.')
            xhi_tmp = SIM_MPCH_LEN
        if yhi_tmp > SIM_MPCH_LEN:
            print(f'Trimming y-high boundary from {yhi_tmp} Mpc/h to sim length.')
            yhi_tmp = SIM_MPCH_LEN
        area = (xhi_tmp - xlow_tmp) * (yhi_tmp - ylow_tmp)
        
        massive_halo_cnt = np.sum(np.logical_and.reduce([full_halo_cat.x_snap >= xlow_tmp,
                                                        full_halo_cat.x_snap < xhi_tmp,
                                                        full_halo_cat.y_snap >= ylow_tmp,
                                                        full_halo_cat.y_snap < yhi_tmp]))
        massive_halo_counts.append(massive_halo_cnt)
        massive_halo_areal_dens.append(massive_halo_cnt / area)
        
        gal_indices = np.array(range(ngal)) * nmocks + mock_ctr
        abs_indices = np.array(range(nabs)) * nmocks + mock_ctr

        xcorr_suffixes = [f'g{g:03d}_a{a:03d}' for g in gal_indices for a in abs_indices]
        xcorrs = np.array([np.load(os.path.join(MOCK_XCORR_DIR, 
                                                f"xcorrmock_{SURVEY_NAME}_{suffix}_{constants.DATA_VERSION}.npy")) 
                           for suffix in xcorr_suffixes])

        xcorrs = np.sum(xcorrs, axis=2)
        assert xcorrs.shape[1] == len(SigEdges) - 1
        
        # Sigma bins greater than ~15 Mpc.
        xcorrs = xcorrs[:, 6:]
        xcorrs = np.sum(xcorrs, axis=1)

        cont_xcorr_means.append(np.mean(xcorrs))
        cont_xcorr_stds.append(np.std(xcorrs))
        
        mock_ctr += 1

In [ ]:
x = massive_halo_counts

plt.errorbar(x, cont_xcorr_means, yerr=cont_xcorr_stds, ls='None', marker='.', capsize=5)
slope, intercept, r_value, _, _ = scipy.stats.linregress(x, cont_xcorr_means)
line_fit_x = np.linspace(np.min(x), np.max(x), 100)
plt.plot(line_fit_x, slope * line_fit_x + intercept, color='black')

plt.xlabel(f'# halos above {HALO_MASS_THRESHOLD:.2e} Msun/h')
plt.ylabel(r'Sum of cross-correlation with $\sigma$ > 15 Mpc')
plt.title(f'Survey: {SURVEY_NAME} | $R^2$ = {r_value**2}')